In [1]:
%pip install dowhy
%apt install libgraphviz-dev
%pip install pygraphviz
%pip install pandas

You should consider upgrading via the '/home/natalia/wsi/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%apt` not found.


In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"

df = pd.read_csv(url)

print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [ ]:
from dowhy import CausalModel
import dowhy.datasets

df_subset = df[['Age', 'Pclass', 'Sex', 'Survived']]
df_cleaned = df_subset.dropna(subset=['Age', 'Pclass', 'Sex', 'Survived'], how='any')


model = CausalModel(
    data=df_cleaned,
    treatment='Age',
    outcome='Survived',
    common_causes=['Pclass', 'Sex'])

identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)

estimate = model.estimate_effect(identified_estimand,
                                 method_name="backdoor.linear_regression")

print(estimate)

*** Causal Estimate ***

## Identified estimand
Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
  d                           
──────(E[Survived|Pclass,Sex])
d[Age]                        
Estimand assumption 1, Unconfoundedness: If U→{Age} and U→Survived then P(Survived|Age,Pclass,Sex,U) = P(Survived|Age,Pclass,Sex)

## Realized estimand
b: Survived~Age+Pclass+Sex
Target units: ate

## Estimate
Mean value: -0.0054531093651492



In [ ]:
import pandas as pd
import dowhy
from dowhy import CausalModel


model = CausalModel(
    data=df,
    treatment='Age',
    outcome='Survived',
    graph="""digraph {
    Age -> Survived
    Sex -> Survived
    Pclass -> Survived
    SibSp -> Survived
    Parch -> Survived
    Embarked -> Survived
}"""
)

identified_estimand = model.identify_effect()

estimate = model.estimate_effect(identified_estimand, method_name="backdoor.propensity_score_matching")

refuted_estimate = model.refute_estimate(identified_estimand, estimate, method_name="random_common_cause")

print(refuted_estimate)


Exception: No common causes/confounders present. Propensity score based methods are not applicable